In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
# ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132243
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-132243


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
param_sampling = RandomParameterSampling( {
        "--C": uniform(0.05, 0.5),
        "--max_iter": choice(range(1,100))
    }
)

# Specify a Policy
policy = policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
              compute_target=compute_target,
              entry_script='train.py') ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=param_sampling,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4) ### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run = exp.submit(hyperdrive_config)

In [5]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

joblib.dump(value = best_run.id, filename='outputs/best_hyperdrive_model.joblib')

best_run.register_model(model_name = 'best_model',model_path='outputs/best_hyperdrive_model.joblib')
print(best_run.get_file_names()[-1])

['--C', '0.08304708356076099', '--max_iter', '16']
outputs/best_hyperdrive_model.joblib


# Auto ML

In [10]:
from train import clean_data
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# datastore_path ="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
# ds = TabularDatasetFactory.from_delimited_files(datastore_path)
datastore_path ="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds =  Dataset.Tabular.from_delimited_files(path=datastore_path)
training_data, validation_data = ds.random_split(percentage=0.8, seed=223)
label_column_name = 'y'

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name=label_column_name,
    n_cross_validations=5)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_1fd583e0-8eea-42a0-8bac-fe0af31ff743

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data ha

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
run.get_output()

(Run(Experiment: udacity-project,
 Id: AutoML_1fd583e0-8eea-42a0-8bac-fe0af31ff743_24,
 Type: azureml.scriptrun,
 Status: Completed),
 PipelineWithYTransformations(Pipeline={'memory': None,
                                        'steps': [('datatransformer',
                                                   DataTransformer(enable_dnn=None,
                                                                   enable_feature_sweeping=None,
                                                                   feature_sweeping_config=None,
                                                                   feature_sweeping_timeout=None,
                                                                   featurization_config=None,
                                                                   force_text_dnn=None,
                                                                   is_cross_validation=None,
                                                                   is_onnx_compatible=Non

In [14]:
best_automl_run, best_model = run.get_output()
best_automl_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-132243', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-132243'), name=best_run_automl.pkl, id=best_run_automl.pkl:1, version=1, tags={}, properties={})

In [15]:

#Take a look to the parameters of the best run
best_automl_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":4}',
 'ensembled_iterations': '[1, 0, 16, 14, 11, 6]',
 'ensembled_algorithms': "['XGBoostClassifier', 'LightGBM', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier']",
 'ensemble_weights': '[0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.07692307692307693, 0.07692307692307693, 0.15384615384615385]',
 'best_individual_pipeline_score': '0.9150268202952155',
 'best_individual_iteration': '1',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [16]:

#Also take a look to the recommended function
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['1', '0', '16', '14', '11', '6'],
 'weights': [0.15384615384615385,
             0.23076923076923078,
             0.3076923076923077,
             0.07692307692307693,
             0.07692307692307693,
             0.15384615384615385]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_stat